In [ ]:
import PyPDF2
import tabula as tb
import pandas as pd
import requests
import ScrapingPrefect as scraping
import re

In [ ]:
def ScrapePDF(sourcepdf):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64)'}
    response = requests.request("GET", sourcepdf,headers=headers)
    file = open("myfile.pdf", "wb")
    file.write(response.content)
    n = PyPDF2.PdfFileReader(open("myfile.pdf",mode='rb')).getNumPages()
    data = tb.read_pdf("myfile.pdf",  pages = f'2-{n}', lattice=True)
    file.close()
    data2 = []
    for i in data:
        i2=i.drop(0)
        data2.append(i2)
    final = pd.concat(data2)
    final = final.rename(columns={
                    final.columns[0]:"Quartier",
                    final.columns[1]:"Num",
                    final.columns[2]:"Désignation",
                    final.columns[3]:"Adresse",
                    final.columns[4]:"Zone"
    }
                )
    final=final.reset_index()
    final=final.drop(["index"],axis=1)
    regex = "([0-9]{1,3}|#####)( [a-z]+|[a-z]+\b(?<!au))?au([0-9]{1,3}| [0-9]{1,3}|#####)( [a-z]+|[a-z]+)?"
    final["Zone"]=final["Zone"].str.replace("\r", " ")
    final2 = final["Zone"].str.extract(regex, expand=True)
    final2 = final2.fillna("")
    final["Zone1"],final["Zone2"]=(
    (final2[final2.columns[0]]+final2[final2.columns[1]]),
    (final2[final2.columns[2]]+final2[final2.columns[3]])
    )
    final["ZoneAdresse"]=final["Zone"].str.replace(regex, "",regex=True)
    final["ZoneAdresse"]=final["ZoneAdresse"].str.replace("\r", " ")
    final["Désignation"]=final["Désignation"].str.replace("\r", " ")
    final["Quartier"]=final["Quartier"].str.replace("\r", " ")
    final["Adresse"]=final["Adresse"].str.replace("\r", " ")
    basefinale = final.drop(["Zone"],axis=1)
    return basefinale  

In [ ]:
last=[]
for i in scraping.listurl("https://www.prefectures-regions.gouv.fr/ile-de-france/Region-et-institutions"
                 "/Demarches-administratives/Elections/Les-bureaux-de-vote-a"
                 "-Paris-periode-du-1er-janvier-2022-au-31-decembre-2022"):
    last.append(ScrapePDF("https://www.prefectures-regions.gouv.fr"+i))
    print(i)

In [ ]:
ListeBDV = pd.concat(last)
ListeBDV['ZoneAdresse'] = ListeBDV['ZoneAdresse'].str.strip()
ListeBDV = ListeBDV.reset_index()
ListeBDV.loc[(ListeBDV.Adresse == '15 RUE ANTOINE CHANTIN'),'Adresse']='15 RUE ANTOINE CHANTIN 75014 PARIS'
ListeBDV["Arrondissement"] = ListeBDV["Adresse"].str.extract("750([0-9]{2})", expand=True)
#ListeBDV.to_csv("ListeBDVParisv4.csv")

In [ ]:
#ListeBDV = pd.read_csv("ListeBDVParisv4.csv")
ListeCirc = pd.read_excel("LEGISLATIVES - Tableau récapitulatif des bureaux de vote et des emplacement d'affichage électoral 2022.xlsx", index_col=0, sheet_name='CIRC_1')
ListeCirc["Circonscription"]=1

In [ ]:
for i in range (2, 19):
    ListeCircTemp=pd.read_excel("LEGISLATIVES - Tableau récapitulatif des bureaux de vote et des emplacement d'affichage électoral 2022.xlsx", index_col=0, sheet_name=f'CIRC_{i}')
    ListeCircTemp["Circonscription"]=i
    ListeCirc = pd.concat([ListeCirc,ListeCircTemp])

In [ ]:
ListeCirc.loc[((ListeCirc["Circonscription"] == 7) & (ListeCirc["Adresse du (ou des) bureau(x) de vote concerné(s)"]== "ECOLE ELEMENTAIRE\n165 RUE DE BERCY 75012 PARIS")),'ARDT']=12.0 #correction Erreur
ListeCirc.loc[((ListeCirc["Circonscription"] == 7) & (ListeCirc["Adresse du (ou des) bureau(x) de vote concerné(s)"]== "MAIRIE D'ARRONDISSEMENT\n12 PLACE LEON BLUM 75011 PARIS")),'ARDT']=11.0 #correction Erreur
ListeCirc[ListeCirc.index!=ListeCirc["Circonscription"]]#contrôle

In [ ]:
ListeCirc2 = ListeCirc.reset_index()
ListeCirc2 = ListeCirc2.drop(["Circ."], axis=1)
ListeCirc3 = ListeCirc2[ListeCirc2["ARDT"].notnull() & ListeCirc2["Bureau(x) de vote concerné(s)"].notnull()]
ListeCirc3[ListeCirc3["ARDT"].isna()]#contrôle

In [ ]:
ListBdvCirc = [0]
for i in range(1,19):
    ListeCirc3t = ListeCirc3[ListeCirc3["Circonscription"]==i][["ARDT","Bureau(x) de vote concerné(s)"]]
    DictBdvTemp = {}
    for j in ListeCirc3t["ARDT"].unique():
        BdvTemp = [re.findall('([0-9]+)',x) for x in ListeCirc3t[ListeCirc3t["ARDT"]==j]["Bureau(x) de vote concerné(s)"]]
        BdvTemp2 = [x for xs in BdvTemp for x in xs]
        DictBdvTemp[int(j)] = BdvTemp2
    ListBdvCirc.append(DictBdvTemp) #Liste de circonscriptions contenant des dictionnaires d'arrondissements contenant la liste des BDV (en désordre)

In [ ]:
def ArrBdvToCirc(Arr,Bdv):
    for i in range(1,19):
        if int(Arr) in ListBdvCirc[i].keys():
            if Bdv in ListBdvCirc[i][int(Arr)]:
                return i
    return "Erreur"

ListeBDV["Circonscription"]=ListeBDV.apply(lambda x: ArrBdvToCirc(x['Arrondissement'], x['Num']), axis=1)

In [ ]:
ListeBDV = ListeBDV.reset_index()
ListeBDV = ListeBDV.drop(["index","level_0"],axis=1)
ListeBDV.to_csv("ListeBDVParisv5.csv")